<a href="https://colab.research.google.com/github/Praxis-QR/RDWH/blob/main/PG24_2D_AmazonSalesData.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

![alt text](https://github.com/Praxis-QR/RDWH/raw/main/images/YantraJaalBanner.png)<br>


<hr>

[Prithwis Mukerjee](http://www.linkedin.com/in/prithwis)<br>

In [1]:
from datetime import datetime
import pytz
print('ॐ श्री सरस्वत्यै नमः',datetime.now(pytz.timezone('Asia/Calcutta')))
!python --version
!lsb_release -a

ॐ श्री सरस्वत्यै नमः 2025-10-23 10:38:33.547357+05:30
Python 3.12.12
No LSB modules are available.
Distributor ID:	Ubuntu
Description:	Ubuntu 22.04.4 LTS
Release:	22.04
Codename:	jammy


In [2]:
# Install postgresql server

!apt update > /dev/null
!apt install postgresql > /dev/null
#
import psycopg2
import pandas as pd
#
!psql -V
!service postgresql start



W: Skipping acquire of configured file 'main/source/Sources' as repository 'https://r2u.stat.illinois.edu/ubuntu jammy InRelease' does not seem to provide it (sources.list entry misspelt?)


psql (PostgreSQL) 14.19 (Ubuntu 14.19-0ubuntu0.22.04.1)
 * Starting PostgreSQL 14 database server
   ...done.


In [3]:
# This has created an Ubuntu user 'postgres' AND a Postgres database user 'postgres' both of which are superusers
#!cat /etc/passwd
# Use these users to create a new database user called 'upraxis' and give it a password 'upass'
#!sudo -u postgres createuser --superuser upraxis
!sudo -u postgres createuser upraxis
!sudo -u postgres psql -U postgres -c "ALTER USER upraxis PASSWORD 'upass';"
# Change the database superuser 'postgres' to a non-default value 'pass'
!sudo -u postgres psql -U postgres -c "ALTER USER postgres PASSWORD 'pass';"
# Use ubuntu superuser to create database dbpraxis and set its owner to upraxis
!sudo -u postgres createdb -O upraxis dbpraxis
# Create file to store the password. Else will be prompted each time
!echo "localhost:5432:dbpraxis:postgres:pass" > ~/.pgpass
!echo "localhost:5432:dbpraxis:upraxis:upass" >> ~/.pgpass
!chmod 0600 ~/.pgpass
!cat ~/.pgpass

ALTER ROLE
ALTER ROLE
localhost:5432:dbpraxis:postgres:pass
localhost:5432:dbpraxis:upraxis:upass


In [4]:
# Create Fact Table using command line interface
#
!psql -h localhost -p 5432 -Uupraxis -ddbpraxis -c \
"drop table if exists amazon;\
CREATE TABLE amazon ( \
    order_index BIGINT PRIMARY KEY, \
    order_id VARCHAR(30) NOT NULL, \
    order_date DATE, \
    status VARCHAR(50), \
    fulfilment VARCHAR(20), \
    sales_channel VARCHAR(20), \
    ship_service_level VARCHAR(20), \
    style VARCHAR(50), \
    sku VARCHAR(100), \
    category VARCHAR(50), \
    size VARCHAR(10), \
    asin VARCHAR(10), \
    courier_status VARCHAR(50), \
    quantity INTEGER, \
    currency VARCHAR(5), \
    amount NUMERIC(10, 2), \
    ship_city VARCHAR(100), \
    ship_state VARCHAR(100), \
    ship_postal_code VARCHAR(20), \
    ship_country VARCHAR(5), \
    promotion_ids TEXT, \
    b2b BOOLEAN, \
    fulfilled_by VARCHAR(100), \
    unnamed_22 TEXT \
);\
"

NOTICE:  table "amazon" does not exist, skipping
CREATE TABLE


In [5]:
#Describe the table ss_order
!psql -h localhost -p 5432 -Uupraxis -ddbpraxis -c '\d+ amazon'

                                                       Table "public.amazon"
       Column       |          Type          | Collation | Nullable | Default | Storage  | Compression | Stats target | Description 
--------------------+------------------------+-----------+----------+---------+----------+-------------+--------------+-------------
 order_index        | bigint                 |           | not null |         | plain    |             |              | 
 order_id           | character varying(30)  |           | not null |         | extended |             |              | 
 order_date         | date                   |           |          |         | plain    |             |              | 
 status             | character varying(50)  |           |          |         | extended |             |              | 
 fulfilment         | character varying(20)  |           |          |         | extended |             |              | 
 sales_channel      | character varying(20)  |      

In [6]:
# original data from https://www.kaggle.com/datasets/thedevastator/unlock-profits-with-e-commerce-sales-data
#
!wget -q -O AmazonSalesData.csv "https://github.com/Praxis-QR/RDWH/raw/refs/heads/main/data/AmazonSalesData.csv"
!wget -q -O AmazonSalesData_Sample.csv "https://github.com/Praxis-QR/RDWH/raw/refs/heads/main/data/AmazonSalesData_Sample.csv"

In [7]:
!psql -h localhost -p 5432 -U postgres -d dbpraxis -c " truncate amazon"
# Load Data
# Need to use U = postgres and provide password = 'pass'
#!psql -h localhost -p 5432 -U postgres -d dbpraxis -c \
#"copy amazon FROM '/content/AmazonSalesData_Sample.csv' DELIMITER ',' CSV HEADER;"

!psql -h localhost -p 5432 -U postgres -d dbpraxis -c \
"copy amazon FROM '/content/AmazonSalesData.csv' DELIMITER ',' CSV HEADER;"

TRUNCATE TABLE
COPY 128975


In [8]:
!psql -h localhost -p 5432 -U upraxis -d dbpraxis -c 'select * from amazon limit 25'

 order_index |      order_id       | order_date |            status            | fulfilment | sales_channel | ship_service_level |  style   |            sku            |   category    | size |    asin    | courier_status | quantity | currency | amount  |    ship_city    |   ship_state   | ship_postal_code | ship_country |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                     

In [10]:
!psql -h localhost -p 5432 -U upraxis -d dbpraxis -c 'select count(*) from amazon'

 count  
--------
 128975
(1 row)



In [11]:
!psql -h localhost -p 5432 -U upraxis -d dbpraxis -c 'select distinct(category) from amazon'

   category    
---------------
 Blouse
 Western Dress
 Saree
 Dupatta
 Bottom
 Ethnic Dress
 Top
 Set
 kurta
(9 rows)



In [13]:
!psql -h localhost -p 5432 -U upraxis -d dbpraxis -c 'select count(distinct(style)) from amazon'

 count 
-------
  1377
(1 row)

